In [1]:
from github2pandas.issues import Issues
from github2pandas.pull_requests import PullRequests
from github2pandas.version import Version
from github2pandas.workflows import Workflows
from github2pandas.utility import Utility
from github2pandas.git_releases import GitReleases
from pathlib import Path
import pandas as pd
import os
import sys
sys.path.append('../')
from src.DataHandling import DataHandling
from src.Utilities import getrepofolder, getTeamName, getTaskName

In [2]:
project_folder_name = "../data/"
project_parameter_file = "parameter.yml"

dataHandler = DataHandling(project_folder_name, project_parameter_file)
github_token = os.environ['TOKEN']

Loading parameter file ... ../data/parameter.yml


In [3]:
relevant_repos = Utility.get_repos(github_token, 
                                    dataHandler.basic_folder,
                                    dataHandler.whitelist_pattern,
                                    dataHandler.blacklist_pattern)

print("{} repos found!".format(len(relevant_repos)))

17 repos found!


In [4]:
raw_pdUsers = pd.DataFrame()
raw_pdCommits = pd.DataFrame()
raw_pdEdits = pd.DataFrame()
raw_pdReleases = pd.DataFrame()

for repo in relevant_repos:
    repo_folder = getrepofolder(repo, dataHandler.basic_folder)
    Version.no_of_proceses = 4
    Version.clone_repository(repo=repo, data_root_dir=repo_folder, github_token=github_token)
    Version.generate_version_pandas_tables(repo = repo, data_root_dir=repo_folder)

    team_users = Utility.get_users(repo_folder)
    team_users['team'] = getTeamName(repo, dataHandler.whitelist_pattern)
    raw_pdUsers = raw_pdUsers.append(team_users, ignore_index=True)
    
    team_commits = Version.get_version(repo_folder)
    team_commits['team'] = getTeamName(repo, dataHandler.whitelist_pattern)
    team_commits['task'] = getTaskName(repo)
    raw_pdCommits = raw_pdCommits.append(team_commits, ignore_index=True)
    
    team_edits = Version.get_version(repo_folder, Version.VERSION_EDITS)
    team_edits['team'] = getTeamName(repo, dataHandler.whitelist_pattern)
    team_edits['task'] = getTaskName(repo)
    raw_pdEdits = raw_pdEdits.append(team_edits, ignore_index=True)
    
    GitReleases.generate_git_releases_pandas_tables(repo, repo_folder)
    git_releases = GitReleases.get_git_releases(repo_folder) 
    git_releases['team'] = getTeamName(repo, dataHandler.whitelist_pattern)
    git_releases['task'] = getTaskName(repo)
    raw_pdReleases = raw_pdReleases.append(git_releases, ignore_index=True)


 -> An exception occurred
Found no database on provided path. Starting from scratch.


Parallel (4 processes): 100%|██████████| 9/9 [00:00<00:00, 15.24it/s]


1
 -> An exception occurred
Found no database on provided path. Starting from scratch.


Parallel (4 processes): 100%|██████████| 17/17 [00:00<00:00, 20.96it/s]


1
Found no database on provided path. Starting from scratch.


...

1
 -> An exception occurred
Found no database on provided path. Starting from scratch.


Parallel (4 processes): 100%|██████████| 8/8 [00:00<00:00, 15.15it/s]


1
Found no database on provided path. Starting from scratch.


Parallel (4 processes): 100%|██████████| 5/5 [00:00<00:00, 17.29it/s]


1


In [5]:
raw_pdUsers.drop(["email"], axis = 1, inplace = True)
raw_pdUsers.drop_duplicates(subset=['anonym_uuid', 'id', 'login', 'team'], inplace = True)
raw_pdCommits.drop(["committer_timezone"], axis = 1, inplace = True)

dataHandler.pickle_File_To_Raw_Data_Folder(raw_pdCommits, "raw_pdCommits.p")
dataHandler.pickle_File_To_Raw_Data_Folder(raw_pdUsers, "raw_pdUsers.p")
dataHandler.pickle_File_To_Raw_Data_Folder(raw_pdReleases, "raw_pdReleases.p")
dataHandler.pickle_File_To_Raw_Data_Folder(raw_pdEdits, "raw_pdEdits.p")